![Logo](../logos/banner_colibri.png)

---
# 🏝️ Welcome to **colibri**'s playground!

Here, you will be able to test all the features of the package. Adapt parameters and run the cells corresponding to the function you want to test to see the result. Make sure you activate the conda environment `colibri` and the Jupyter kernel before starting. A quick description of the functionalities is provided. More technical information is available in docstrings of each function.

---
<br/>
<br/>

In [3]:
# Import colibri

import sys

sys.path.append("..")
import src

AttributeError: partially initialized module 'src' has no attribute 'scrapper' (most likely due to a circular import)


<br/>

**1. Run the entire pipeline** <br/>
xxx

In [2]:
query = "ts = (('meta*analysis' or 'systematic review') and ('soil organic carbon' or 'SOC' or 'soil organic matter' or 'SOM' or 'soil carbon'))"
src.wrapper.run_pipeline(query, ["WoS"])

NameError: name 'src' is not defined

<br/>

**2. Web of Science scrapping** <br/>
Scrape publications on Web of Science (Core Collection) with a specific search query. DOIs, titles, abstracts and keywords (when available) of each publication will be stored in a Pickle file `colibri/data/wos/<yyyy>-<mm>-<dd>_<hh>-<mm>-<ss>` with the corresponding timestamp of your scrapping.

In [ ]:
# query = "ts = (('meta*analysis' or 'systematic review') and ('soil organic carbon' or 'SOC' or 'soil organic matter' or 'SOM' or 'soil carbon'))"
# src.scrapper.scrapper(query, ["WoS"])